In [ ]:
import torch
import torch.nn as nn
import tiktoken
from torch.utils.data import Dataset, DataLoader

from llm_from_scratch.previous_chapters import generate, load_weights_into_gpt, text_to_token_ids, token_ids_to_text
from gpt_download import *

GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.0,
    "qkv_bias": False
}


class MyDataset(Dataset):
    def __init__(self, token_ids, context_length, stride):
        super().__init__()
        self.input_dataset = []
        self.target_dataset = []

        for i in range(0, len(token_ids) - context_length, stride):
            self.input_dataset.append(torch.tensor(token_ids[i:i + context_length]))
            self.target_dataset.append(torch.tensor(token_ids[i + 1:i + context_length + 1]))

    def __len__(self):
        return len(self.input_dataset)

    def __getitem__(self, idx):
        return self.input_dataset[idx], self.target_dataset[idx]


def create_dataloader(raw_text, batch_size, context_length, stride, shuffle=True, drop_last=True, num_processor=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    token_ids = tokenizer.encode(raw_text, allowed_special={"|<endoftext>|"})
    dataset = MyDataset(token_ids, context_length, stride)
    dataloader = DataLoader(
        dataset=dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_processor, drop_last=drop_last
    )
    return dataloader


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))


class LayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x: torch.Tensor):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        return self.scale * ((x - mean) / torch.sqrt(var + self.eps)) + self.shift


class FeedForward(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(emb_dim, 4 * emb_dim),
            GELU(),
            nn.Linear(4 * emb_dim, emb_dim)
        )

    def forward(self, x):
        return self.layers(x)


class MyAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        batch_size, context_length, emb_dim = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        queries = queries.view(batch_size, context_length, self.num_heads, self.head_dim)
        keys = keys.view(batch_size, context_length, self.num_heads, self.head_dim)
        values = values.view(batch_size, context_length, self.num_heads, self.head_dim)

        queries = queries.transpose(1, 2)
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)

        context_scores = queries @ keys.transpose(2, 3)

        context_scores.masked_fill_(self.mask.bool()[:context_length, :context_length], -torch.inf)

        # context_weight = torch.softmax(context_scores / emb_dim ** 0.5, dim=-1)   emb_dim is wrong, after view function you should divided by self.head_dim
        context_weight = torch.softmax(context_scores / self.head_dim ** 0.5, dim=-1)
        context_weight = self.dropout(context_weight)

        context_vec = (context_weight @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(batch_size, context_length, emb_dim)
        context_vec = self.out_proj(context_vec)  # optional
        return context_vec


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MyAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg["emb_dim"])
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_resid = nn.Dropout(cfg['drop_rate'])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_resid(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut
        return x


class GPTMODEL(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.dropout = nn.Dropout(cfg['drop_rate'])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'], cfg['vocab_size'], bias=False)
        self.context_size = cfg['context_length']

    def forward(self, token_ids):
        context_length = token_ids.shape[1]
        token_embeddings = self.tok_emb(token_ids)
        pos_embeddings = self.pos_emb(torch.arange(context_length, device=token_ids.device))
        x = token_embeddings + pos_embeddings
        x = self.dropout(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.context_size:]
            logits = self(idx_cond)
            logits = logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


def get_loss_batch(dataloader, num_dataloader, model: GPTMODEL):
    model.eval()
    loss = 0
    current_num = 0
    for num, (inputs, targets) in enumerate(dataloader):
        logits = model(inputs)
        current_num = num
        loss += torch.nn.functional.cross_entropy(logits.flatten(0, 1), targets.flatten(0, 1))
        if num >= num_dataloader:
            break
    model.train()
    return loss / (current_num + 1)


def predicts(model, need_predict_token_num, start_text, device, context_size, tokenizer):
    model.eval()
    token_ids = tokenizer.encode(start_text, allowed_special={'<|endoftext|>'})
    token_ids_tensor = torch.tensor(token_ids, device=device).unsqueeze(0)
    with torch.no_grad():
        for _ in range(need_predict_token_num):
            token_ids_tensor = token_ids_tensor[:, -context_size:]
            logits = model(token_ids_tensor)
            logits = logits[:, -1, :]
            probas = torch.softmax(logits, dim=-1)
            id_next = torch.argmax(probas, dim=-1, keepdim=True)
            token_ids_tensor = torch.cat((token_ids_tensor, id_next), dim=-1)
    decoded_text = tokenizer.decode(token_ids_tensor.squeeze(0).tolist())
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

training_ratio = 0.9
training_indice = int(len(raw_text) * 0.9)
training_text = raw_text[:training_indice]
val_text = raw_text[training_indice:]
torch.manual_seed(123)
training_loader = create_dataloader(training_text, 2, GPT_CONFIG_124M['context_length'],
                                    GPT_CONFIG_124M['context_length'], True, True, 0)
val_loader = create_dataloader(val_text, 2, GPT_CONFIG_124M['context_length'],
                               GPT_CONFIG_124M['context_length'], False, False, 0)
model = GPTMODEL(GPT_CONFIG_124M)

In [ ]:
torch.manual_seed(123)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.train()
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
global_step = -1
tokenizer = tiktoken.get_encoding("gpt2")
for epoch in range(10):
    for inputs, targets in training_loader:
        inputs: torch.Tensor
        targets: torch.Tensor
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)
        logits = model(inputs)
        loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), targets.flatten(0, 1))
        loss.backward()
        optimizer.step()
        global_step += 1

        if global_step % 5 == 0:
            eval_loss = get_loss_batch(val_loader, 5, model)
            train_loss = get_loss_batch(training_loader, 5, model)
            print(f"epoch: {epoch}, step: {global_step} \n"
                  f"validation loss:{eval_loss:.2f}, training loss: {train_loss:.2f}")

    predicts(model, 50, "Every effort moves you", device, model.pos_emb.weight.shape[0], tokenizer)

In [ ]:
get_loss_batch(val_loader, 5, model)

In [ ]:
# Save and Load
torch.save(
    {"model_state_dict": model.state_dict(),
     "optimizer_state_dict": optimizer.state_dict()
     },
    "model_and_optimizer.pth"
)
checkpoint = torch.load("model_and_optimizer.pth")
model = GPTMODEL(GPT_CONFIG_124M)
model.load_state_dict(checkpoint['model_state_dict'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = tiktoken.get_encoding("gpt2")
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.1)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

In [ ]:
model_config = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

model_name = "gpt2-small (124M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_config[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt = GPTMODEL(NEW_CONFIG)
load_weights_into_gpt(gpt, params)

In [ ]:
torch.manual_seed(123)
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate(model=gpt, idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
                     max_new_tokens=25,
                     context_size=NEW_CONFIG['context_length'], top_k=50,
                     temperature=1.4)
print(token_ids_to_text(token_ids, tokenizer))

In [ ]:
from llm_from_scratch.previous_chapters import generate_text_simple

tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(model=gpt, idx=text_to_token_ids("Every effort moves you", tokenizer),
                                 max_new_tokens=15, context_size=NEW_CONFIG['context_length'])
print(token_ids_to_text(token_ids, tokenizer))

In [ ]:
%%sql


In [ ]:
from llm_from_scratch.previous_chapters import GPTModel
import torch


def compare_models(model1, model2, atol=1e-6, rtol=1e-5):
    """
    Compare two PyTorch models to ensure all parameters match.

    Args:
        model1 (torch.nn.Module): First model to compare.
        model2 (torch.nn.Module): Second model to compare.
        atol (float): Absolute tolerance for weight comparison.
        rtol (float): Relative tolerance for weight comparison.

    Returns:
        bool: True if models are identical, False otherwise.
    """
    # Ensure models have the same number of parameters
    params1 = dict(model1.named_parameters())
    params2 = dict(model2.named_parameters())

    if len(params1) != len(params2):
        print(f"Parameter count mismatch: {len(params1)} vs {len(params2)}")
        return False

    identical = True
    for name in params1:
        if name not in params2:
            print(f"Missing parameter in model2: {name}")
            identical = False
            continue

        param1 = params1[name].detach().cpu()
        param2 = params2[name].detach().cpu()

        # Check shape
        if param1.shape != param2.shape:
            print(f"Shape mismatch in {name}: {param1.shape} vs {param2.shape}")
            identical = False
            continue

        # Check values
        if not torch.allclose(param1, param2, atol=atol, rtol=rtol):
            print(f"Value mismatch in {name}")
            identical = False

    return identical


CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,  # Dropout rate
    "qkv_bias": True  # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
token_ids = generate_text_simple(model=model, idx=text_to_token_ids("Every effort moves you", tokenizer),
                                 max_new_tokens=15, context_size=BASE_CONFIG['context_length'])
print(token_ids_to_text(token_ids, tokenizer))

In [ ]:
# compare_models(model, gpt)
gpt.eval()
token_ids = generate_text_simple(model=gpt, idx=text_to_token_ids("Every effort moves you", tokenizer),
                                 max_new_tokens=15, context_size=BASE_CONFIG['context_length'])
print(token_ids_to_text(token_ids, tokenizer))

In [ ]:
context_size = 1024
idx = text_to_token_ids("Every effort moves you", tokenizer)
with torch.no_grad():
    idx_cond = idx[:, -context_size:]
    logits1 = gpt(idx_cond)[:, -1, :]
    logits2 = model(idx_cond)[:, -1, :]

    print("Same logits?", torch.allclose(logits1, logits2, atol=1e-6))

In [6]:
import torch

print(torch.triu(torch.ones(5, 4), diagonal=1))

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
